In [1]:
import sys
sys.path.append("C:/Users/Tobias.Turke/Desktop/Uni/GraphToSequenceNN/Scripts/")
import numpy as np
from keras import backend as K
from keras.models import Model
from keras import regularizers, activations
from keras.layers import Lambda, concatenate, Dense, Dropout, Input, Embedding, LSTM

from keras.utils import plot_model
from keras.callbacks import History

from NetworkHandler.KerasSupportMethods.SupportMethods import IsKerasTensor
from NetworkHandler.KerasSupportMethods.SupportMethods import ConsoleReporter
from NetworkHandler.Neighbouring.NeighbourhoodCollector import Neighbourhood as Nhood
from NetworkHandler.KerasSupportMethods.ControledBasicOperations import ControledTensorOperations as CTO
from NetworkHandler.Builder.ModelBuilder import ModelBuilder

'''
Great resources 
    => https://theailearner.com/2019/01/25/multi-input-and-multi-output-models-in-keras/
    => https://machinelearningmastery.com/keras-functional-api-deep-learning/
'''  
hops = 3


Using TensorFlow backend.


In [ ]:
a_sent = "#::snt On the fifth day -- again , as always , it was thanks to the sheep -- the secret of the little prince 's life was revealed to me ."
a_padding = np.array([[-0.04877, -0.085815, 0.3459, 0.23467, -0.24652, -0.0024387
, -0.61645, 0.18601, 0.69535, 0.083516, 0.23424, 0.88833
, 0.6075, 0.13255, 0.072582, 0.35103, 0.34747, 0.14635
, -0.17791, 0.0073774, -0.5488, -0.83556, -0.44298, -0.36361
, -0.48151, -0.29974, 0.13284, -0.45781, 0.24985, -0.35233
, 0.43363, -0.18191, -0.55982, -0.057025, -0.21306, 0.2887
, 0.037381, -0.34767, -0.29849, 0.27515, 0.040138, -0.18286
, -0.18617, 0.079347, -1.0163, 0.097806, 0.022047, 0.38522
, 0.12066, -0.56447, 0.88002, 0.33657, 0.48331, 0.84451
, 0.0697, -1.3009, -0.23264, -0.50579, 0.8919, 0.38409
, -0.27957, 1.3675, -0.24134, -0.074416, 0.98454, -0.23483
, 0.70134, 0.55615, -0.020365, -0.13608, 0.15268, 0.41626
, -0.037752, -0.48355, 0.089359, -0.0486, 0.11818, -0.50971
, -0.52682, 0.27415, 0.13318, 0.10679, -0.027066, -0.18455
, -1.1441, -0.89183, 0.46235, 0.62799, -0.061954, 0.53698
, 0.013082, 0.11236, 0.40615, -0.46954, 0.21148, -0.42949
, -0.70103, -0.64169, -0.19558, 0.0086805],
[0.12728, 0.33918, 0.047297, -0.59545, -0.50807, -0.1497, 0.30002
, 0.28764, -0.45, -0.98081, -1.3502, 0.64971, 0.7544, -0.23777
, 0.80742, 0.59327, 0.41876, 0.070505, 0.026816, 0.2541, 0.72081
, 0.39636, 0.7526, -0.94346, 0.61644, 0.6186, -0.67133, -0.091559
, -0.3541, 0.65173, -0.049964, -0.12267, 0.20019, -0.097971, 0.25479
, 0.26575, -0.23218, 0.53218, -0.055243, -0.252, 0.31309, -0.07482
, -0.045389, -0.30998, -0.20094, -0.10533, 0.55923, -0.25827, -0.54868
, -0.57999, -0.67253, -0.034722, 0.27119, 0.82289, -0.71094, 0.02069
, -1.1679, 0.039353, 0.12002, -0.31662, 1.4502, 0.91076, 1.1657
, -0.21669, 1.3254, 0.8902, -0.43057, -0.77069, 0.90518, -0.38127
, -0.28038, 0.35435, 0.22964, -0.21386, 0.32834, 0.14031, 0.076089
, 0.74345, -0.3713, 0.52666, 0.6998, 0.64682, -0.51838, 0.30539
, -0.67797, -0.3944, 0.31669, 0.2097, 0.46173, 0.64263, -0.1257
, -0.82134, 0.37247, -0.39669, -1.2945, -0.73274, -0.72885, 0.26961
, 0.74588, -0.80255],
[-0.056244, 0.55972, 0.4774, -0.22186, 0.020482, 0.74382, -0.56498
, 0.039568, -0.33427, -0.61848, 0.55888, -0.51747, 0.060808, -0.004491
, -0.35002, -0.5119, -0.13634, -0.16757, -0.64543, 0.78587, -0.29346
, 0.27956, -1.2701, -0.54094, 0.19525, 0.85723, -0.48684, 0.15002
, 1.1267, -0.59077, 0.28363, 0.51272, 0.56589, 0.44085, 0.027534
, 0.11659, -0.24666, 0.2655, 0.1252, -0.73648, 0.015985, 0.31652
, 0.53769, -0.21533, -0.16764, -0.002708, -0.11672, -0.37111, -0.38105
, -1.0881, 0.012516, 0.32336, 0.41341, 0.24932, -0.49713, -0.85652
, 0.24534, 0.25204, 0.56825, -0.23917, 0.028714, 0.5938, -0.67779
, -0.49363, -0.11606, 0.21212, 0.17538, 0.99233, 0.055515, -0.48426
, 0.49162, 0.42797, 0.38566, -1.2318, 0.42758, 1.2102, 0.37246
, 0.19461, -0.24773, 0.067857, 0.13316, -0.26781, -0.09634, -0.39789
, -1.2573, 0.03879, -0.21023, 0.07462, -0.64179, -0.53065, 0.15409
, -0.45997, 0.52257, 0.28125, -0.64359, -0.42087, 0.017241, 0.2782
, -0.68456, 0.53669],
[0.69149, -0.31051, 0.49737, -0.14275, 0.083858, 0.20704
, -0.1459, 0.060682, 0.3124, 0.084921, 0.58973, 0.5637
, 0.26867, 0.92368, -0.47134, 0.61439, 1.3068, -0.020856
, -0.3139, 0.024418, -0.2619, 0.060031, -0.26422, -0.73088
, -0.2843, -0.27944, 0.34297, -0.17607, 0.36038, 0.2627
, 0.41432, -0.057146, -0.22342, -0.63108, 0.22336, 0.48596
, -0.079006, -0.35827, 0.39935, -0.012151, -0.12571, -0.022445
, 0.61281, 0.42427, 0.27414, -0.64438, -0.38072, 0.1139
, 0.38408, -0.03143, 0.8236, 0.31126, 0.60459, 1.0502
, -0.18775, -1.928, 0.15482, 0.13293, 1.4056, 0.13095
, -0.29393, 0.82182, 0.27929, -0.56403, 0.83337, -0.41205
, 0.38721, 0.25896, 0.34559, 0.53165, 0.12766, -0.53753
, -0.11779, -0.32434, -0.5555, 0.631, 0.0094406, -0.50435
, -1.2445, 0.72065, 0.91881, 0.65188, 0.61521, 0.3083
, -1.6975, 0.2664, 0.94071, 0.77578, 0.21495, 0.11198
, -0.41021, 0.10755, 0.33734, 0.37012, 0.16291, -0.53977
, -0.31273, -0.32271, -0.50286, 0.20223],
[0.25157, 0.4589, 0.30274, 0.12461, 0.15062, 0.7373
, -0.31431, -0.31305, -0.40889, 0.042455, -0.42614, 0.49549
, 0.010468, 0.22204, 0.028837, -0.59052, 0.53351, 0.17798
, -0.24486, 0.92695, 0.27058, -0.096303, -0.0037595, 0.065192
, 0.58494, 0.39373, -0.43439, -1.0214, -0.12035, 0.30562
, -0.1696, 0.18668, -0.44006, -0.34578, -0.46857, 0.18542
, -0.30383, 0.49897, -0.37543, -0.52041, -0.5141, -0.13442
, 0.11745, -0.13786, -0.2505, 0.41267, 0.06101, -0.10372
, 0.34358, -0.53381, -0.013572, -0.50581, 0.47017, 1.5464
, 0.21803, -2.2455, 0.35391, -0.12865, 1.587, 0.79569
, 0.15427, 1.262, -0.26174, -0.6863, 1.1039, 0.33912
, 0.88084, -0.24413, 0.21744, -0.21199, 0.09736, -0.17727
, 0.062944, -0.11046, 0.21855, 0.038796, 0.17231, -0.58236
, -0.86605, -0.022036, -0.01176, 0.90198, 0.20412, 0.70519
, -1.5673, 0.11232, -0.35443, -0.73389, -0.28524, -0.74957
, 0.49042, -0.45365, 0.81719, 0.15014, -0.31772, 0.031711
, 0.069947, -0.7478, 0.13684, 0.18927],
[-0.10462, -0.44546, 0.73247, 0.33399, 0.74169, 0.26785, -0.13302
, 0.048104, -0.58442, -0.144, 0.042539, 0.91575, -0.27711, 0.025089
, 0.54384, -0.96325, -0.15653, -1.2196, -0.82354, 0.41574, -0.5213
, -0.43856, 0.15554, -0.33582, -0.26954, 0.28767, -0.44932, 0.12631
, 0.80588, 0.037442, 0.083067, -0.16238, 0.66295, -0.40002, -0.23491
, 0.32218, 0.32082, 0.82943, -0.73548, -0.22967, 0.32303, 0.50304
, 0.42311, -1.0475, 0.50004, -0.26974, -0.67122, 0.34976, 0.84703
, -0.24704, 0.0171, 0.15045, 0.70515, 0.44578, -0.44401, -2.1091
, -0.76305, 0.44046, 0.33391, 0.28334, 0.17089, 0.1954, -0.040206
, 0.27536, 0.71581, -0.16719, -0.12637, 0.53172, -0.53269, 0.45923
, 0.17436, -1.0175, 0.038111, -0.82138, -0.15615, 0.35239, 0.31692
, -0.036803, -0.037231, 0.40949, 0.74874, 0.38309, 0.96702, 0.2672
, -0.73418, -0.23598, -0.67766, -1.1542, 0.58595, -1.0541, 0.23824
, -0.94192, 0.58186, -0.13338, -1.1079, 0.43486, -0.85339, -0.66948
, -0.28375, -1.2942],
[-1.9006e-01, 3.1120e-01, 4.7171e-01, -2.8056e-01, -4.8337e-01, 6.8553e-02
, -4.5067e-01, 1.1173e-01, 6.6101e-02, -7.1041e-01, -2.4859e-01, 1.3370e-01
, -1.3752e-01, -1.5745e-01, 4.0046e-01, 5.2179e-02, -2.0072e-01, -7.5249e-03
, 2.4160e-01, 6.9304e-01, 5.8341e-01, 9.3729e-01, -9.5317e-02, -6.1119e-01
, 1.5453e-01, 2.0619e-01, -5.8002e-01, -8.1164e-01, -2.4534e-01, 1.0759e-01
, -4.7973e-02, 6.4102e-02, 2.2681e-01, -3.3923e-01, -1.5209e-01, 3.1977e-01
, -3.7060e-02, 3.6081e-01, 5.7981e-01, -8.0079e-02, -1.5822e-01, -5.8878e-01
, -6.2072e-02, -7.0349e-01, -2.6627e-01, -2.4602e-01, 4.5015e-01, 2.5519e-01
, 4.8353e-01, -7.6818e-01, -2.2812e-01, -2.9609e-01, 1.1409e-01, 9.1713e-01
, -2.9223e-01, -3.1109e+00, -5.4891e-02, -3.6824e-01, 1.1675e+00, 5.4894e-02
, 1.3308e-01, 1.2725e+00, -1.1196e+00, 3.5718e-01, 6.8855e-01, -5.2246e-01
, 4.9855e-01, 6.1230e-02, 4.1514e-01, -5.0192e-02, -1.1963e-02, -2.6769e-01
, -1.3192e-01, -1.5594e-01, 1.9538e-01, 8.1034e-01, -1.5480e-01, 4.1017e-02
, -5.1727e-01, 1.9496e-01, 4.7015e-01, -1.1347e-01, -1.6959e-01, -7.4851e-03
, -1.5788e+00, -1.7431e-01, 3.3742e-01, 4.5661e-02, -2.1704e-01, -3.9708e-01
, -2.9676e-03, -2.2496e-01, -2.1707e-01, 6.9045e-02, -6.8760e-01, -4.2580e-01
, -5.8658e-01, -5.3615e-01, 6.2931e-02, 6.1613e-01],
[-0.046539, 0.61966, 0.56647, -0.46584, -1.189, 0.44599
, 0.066035, 0.3191, 0.14679, -0.22119, 0.79239, 0.29905
, 0.16073, 0.025324, 0.18678, -0.31001, -0.28108, 0.60515
, -1.0654, 0.52476, 0.064152, 1.0358, -0.40779, -0.38011
, 0.30801, 0.59964, -0.26991, -0.76035, 0.94222, -0.46919
, -0.18278, 0.90652, 0.79671, 0.24825, 0.25713, 0.6232
, -0.44768, 0.65357, 0.76902, -0.51229, -0.44333, -0.21867
, 0.3837, -1.1483, -0.94398, -0.15062, 0.30012, -0.57806
, 0.20175, -1.6591, -0.079195, 0.026423, 0.22051, 0.99714
, -0.57539, -2.7266, 0.31448, 0.70522, 1.4381, 0.99126
, 0.13976, 1.3474, -1.1753, 0.0039503, 1.0298, 0.064637
, 0.90887, 0.82872, -0.47003, -0.10575, 0.5916, -0.4221
, 0.57331, -0.54114, 0.10768, 0.39784, -0.048744, 0.064596
, -0.61437, -0.286, 0.5067, -0.49758, -0.8157, 0.16408
, -1.963, -0.26693, -0.37593, -0.95847, -0.8584, -0.71577
, -0.32343, -0.43121, 0.41392, 0.28374, -0.70931, 0.15003
, -0.2154, -0.37616, -0.032502, 0.8062],
[-3.6689e-01, 4.1540e-01, 1.3478e-01, -1.7841e-01, -4.0050e-01, 2.0647e-01
, 4.2742e-01, 1.1290e+00, -8.3981e-01, -4.7955e-01, 2.1115e-02, -1.8319e-01
, 2.4588e-01, -2.9048e-02, 2.9683e-01, -4.4613e-01, 1.7493e-01, -1.6619e-01
, -4.9573e-01, -2.6547e-02, 7.6369e-01, -2.0810e-01, 2.6494e-01, 3.3085e-01
, 5.7039e-01, 1.2832e-01, -2.8324e-01, 7.8652e-02, 3.8330e-01, -7.4012e-02
, -5.6725e-01, -2.5826e-01, 1.4013e-01, 1.1077e-01, -2.1629e-01, 4.0637e-01
, -4.5740e-01, 2.7936e-01, 1.6717e-01, -1.6388e-01, -6.9184e-01, -3.1493e-01
, 6.5592e-01, -2.2426e-03, -1.6975e-02, 1.0777e-01, -1.8928e-01, -7.7079e-01
, 2.4333e-01, -1.1467e+00, 1.2931e-01, -5.9668e-01, 5.2350e-01, 1.0476e+00
, -8.3517e-01, -2.6268e+00, -3.2837e-01, 2.4767e-01, 2.1535e+00, 6.0561e-01
, -5.4019e-01, 9.9737e-01, -1.8484e-01, -4.7044e-01, -1.9265e-02, 3.4195e-01
, 3.1006e-01, -3.1499e-01, 2.9990e-01, -3.7647e-01, -1.0041e-01, -4.8373e-01
, -3.8982e-01, -3.4424e-01, -1.2195e-01, 5.8945e-01, -1.5675e-01, -3.3774e-01
, -1.0446e+00, -3.7927e-02, 3.8237e-01, -6.3733e-02, -1.2990e-01, 1.2899e-01
, -1.4625e+00, -2.2875e-01, -3.6410e-01, -1.0337e-01, 2.3737e-01, -2.3498e-01
, 1.5801e-01, 3.8339e-02, 1.3424e-02, 3.2535e-01, -1.1755e+00, 4.9695e-02
, 5.7227e-01, 2.4380e-02, 2.2106e-01, 4.3169e-01],
[-0.44967, 0.18737, 0.30528, 1.5052, -0.78491, 0.96247
, 0.21617, 0.02866, -0.16864, 0.033828, -0.3245, -0.747
, -0.52861, 0.4295, 1.0428, -0.5439, -0.43982, 2.0954
, 0.75172, 0.042219, -0.51501, -0.82684, -0.46137, 0.22105
, -0.19863, -0.92268, 0.59337, 0.21477, 0.17061, -0.14858
, 0.71387, -0.40705, -0.29581, 0.13499, 0.67741, -0.22866
, -0.15554, 0.037942, 0.19441, -0.33801, 0.09972, 0.44412
, 0.40887, 0.82193, 0.25378, -0.51247, 0.57974, -0.41434
, -0.37776, 1.023, 0.66726, -0.16978, 0.0021428, -0.28371
, -0.61059, 0.15327, 0.19883, 0.52986, 0.15731, 1.165
, -0.2486, -1.6955, -0.2565, 0.82746, 1.1155, -0.71117
, -0.28927, -0.7435, 0.95892, -0.15997, -0.41062, -0.56523
, 0.26417, -0.88909, 0.67315, -0.023636, -0.37231, -0.78933
, 0.40019, -0.091099, 0.26671, -1.2508, -0.601, -0.99084
, 0.71534, -0.61422, 0.54705, -0.63898, 0.31393, -0.716
, -0.56842, 1.2224, 0.20728, -0.59442, -0.1524, -0.83132
, 0.30632, -0.90266, -0.96749, 0.078221],
[9.3843e-02, -2.0283e-01, 3.4687e-01, -1.6559e-01, 9.3731e-02, 1.9882e-01
, -6.8497e-02, 7.6033e-02, 3.1285e-01, -5.9245e-01, 1.9608e-01, 5.7128e-02
, 7.2130e-02, -2.3307e-02, 1.3256e-01, -4.4215e-01, -2.2331e-01, 5.7460e-01
, -5.6796e-01, -2.9212e-01, 4.8023e-01, -1.9122e-01, 2.8004e-01, 2.3993e-01
, 2.8304e-01, 1.0797e-01, -7.0790e-01, -5.2382e-01, 6.4586e-01, 1.6809e-03
, -2.7394e-01, 6.7346e-01, -2.0844e-01, -2.4083e-03, -7.6281e-02, 3.2356e-01
, -5.3207e-01, -9.3861e-02, -3.9966e-01, -3.9100e-01, -6.3573e-01, -3.3144e-01
, 1.6891e-01, -2.7671e-01, 2.5438e-01, -2.9795e-01, 4.4488e-01, -5.2510e-01
, 4.4147e-01, -1.2168e+00, -1.7280e-01, -1.1873e-02, 2.7402e-01, 1.6268e+00
, -2.3643e-01, -2.8586e+00, -4.3896e-01, -1.3818e-01, 1.0943e+00, 6.9101e-01
, -2.0175e-01, 9.0114e-01, -6.0804e-01, 1.4836e-01, 6.4141e-01, -3.3438e-02
, 2.5421e-01, 3.6123e-01, -1.4983e-01, 2.5836e-01, 1.9554e-01, -2.1860e-01
, -1.2491e-01, -4.3070e-01, 2.2501e-01, -2.7376e-02, -3.4659e-01, 7.4732e-02
, -3.7176e-01, 6.4736e-03, 3.2609e-01, -1.0519e-01, -6.2855e-01, 1.8491e-01
, -7.7250e-01, -6.1363e-01, -7.9837e-02, -7.1546e-02, -5.5032e-01, 6.4136e-02
, -4.8826e-01, -1.9635e-01, -2.4152e-01, 7.3230e-01, -5.5225e-01, 2.4745e-01
, 7.0344e-02, 5.8733e-01, 8.4153e-02, 1.0260e-01],
[-0.2879, 0.60679, 0.25496, -0.31434, -0.17251, 0.39788
, -0.40171, 0.11484, -0.020758, -0.49304, 0.0070329, 0.038979
, 0.56994, 0.29172, -0.36511, -0.48048, 0.0198, 0.44609
, -0.34393, 0.31444, 0.18358, 0.40278, -0.1381, -0.7808
, 0.18664, 0.22567, -0.50004, -0.56416, 0.10707, -0.52422
, -0.27279, 0.97225, 0.14121, 0.26459, 0.76572, 0.25785
, -0.11132, 0.1715, 0.27315, -0.58276, -0.11957, -0.013399
, 0.48315, -0.58191, -0.44008, -0.51884, 0.44876, -0.031764
, 0.43263, -1.1944, 0.33327, -0.43381, 0.1105, 1.1919
, 0.265, -2.0789, 0.12749, 0.30327, 0.68232, 0.50884
, -0.033885, 0.6303, -0.63151, -0.26801, 1.0143, -0.0060002
, 0.72146, 0.0027353, -0.21108, -0.11097, 0.32538, -0.48326
, 0.18443, -0.013396, 0.26999, 0.012397, -0.090559, -0.30702
, -0.4749, 0.11389, 0.089547, -0.02269, -0.82589, -0.21511
, -2.0313, -0.38484, -0.55163, -0.31692, -0.26436, -0.44703
, -0.48504, -0.19752, 0.13092, -0.22776, -0.58003, 0.29211
, -0.60933, -0.71908, 0.057364, 1.0433],
[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.
, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.
, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.
, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.
, 0., 0., 0., 0.],
[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.
, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.
, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.
, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.
, 0., 0., 0., 0.],
[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.
, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.
, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.
, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.
, 0., 0., 0., 0.],
[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.
, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.
, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.
, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.
, 0., 0., 0., 0.],
[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.
, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.
, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.
, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.
, 0., 0., 0., 0.],
[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.
, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.
, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.
, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.
, 0., 0., 0., 0.],
[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.
, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.
, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.
, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.
, 0., 0., 0., 0.]], dtype='float32')

a_p_ten = K.variable (value=a_padding, dtype='float32')

a_fw = np.array([[0., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 0., 0., 0., 0.,
        0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.]], dtype='float32')

a_fw_ten = K.variable (value=a_fw, dtype='float32')

a_bw = np.array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.]], dtype='float32')


a_bw_ten = K.variable (value=a_bw, dtype='float32')

b_sent = '#::snt Abruptly , without anything to lead up to it , and as if the question had been born of long and silent meditation on his problem , he demanded : " A sheep -- if it eats little bushes , does it eat flowers , too ? "'
b_padding = np.array([
[0.061834, 0.55929, 0.2585, 0.27757, 0.18372, -1.3936, -1.1094
, -0.18045, 0.47037, 0.089192, -0.40021, -0.18204, 0.36476, -0.074759
, -0.18475, -0.14079, -0.88115, 0.34402, 0.27577, -0.018601, 0.45366
, -0.67533, 0.29098, 0.43438, -0.045076, 0.011948, -0.14841, -0.7185
, 0.088472, 0.32674, 0.34051, -0.024428, -0.68885, -0.062711, -0.22771
, 0.60763, 0.98759, 0.087323, -0.17409, -0.72005, -0.36272, -0.47358
, 0.30662, -0.028084, 0.76042, -0.69563, 0.044731, -0.664, 0.47093
, -1.1978, 0.2954, 0.021524, -0.025086, 1.0511, 0.51078, -1.5337
, -0.14041, -0.39661, 1.9896, -0.29244, 0.11762, -0.41284, -0.63953
, -0.27706, 0.50936, 0.07978, 0.35552, -0.65102, 0.63215, -0.58636
, 0.48778, -0.18998, -0.42767, -0.38906, 0.20675, 0.50484, 0.003872
, 0.28608, -0.77503, -0.39558, 1.1252, -0.050875, -0.84966, -0.033196
, -0.93163, -0.74732, 0.43482, -0.018736, -0.05062, -0.4161, 0.3855
, -0.1027, -0.52056, -0.30861, -0.83593, -0.2802, 0.26511, 0.084794
, 1.176, 0.21967],
[0.1225, -0.058833, 0.23658, -0.28877, -0.028181, 0.31524
, 0.070229, 0.16447, -0.027623, 0.25214, 0.21174, -0.059674
, 0.36133, 0.13607, 0.18755, -0.1487, 0.31315, 0.13368
, -0.59703, -0.030161, 0.080656, 0.26162, -0.055924, -0.35351
, 0.34722, -0.0055801, -0.57935, -0.88007, 0.42931, -0.15695
, -0.51256, 1.2684, -0.25228, 0.35265, -0.46419, 0.55648
, -0.57556, 0.32574, -0.21893, -0.13178, -1.1027, -0.039591
, 0.89643, -0.9845, -0.47393, -0.12855, 0.63506, -0.94888
, 0.40088, -0.77542, -0.35153, -0.27788, 0.68747, 1.458
, -0.38474, -2.8937, -0.29523, -0.38836, 0.94881, 1.3891
, 0.054591, 0.70486, -0.65699, 0.075648, 0.7655, -0.63365
, 0.86556, 0.42441, 0.14796, 0.4156, 0.29354, -0.51295
, 0.19635, -0.45568, 0.0080246, 0.14528, -0.15395, 0.11406
, -1.2167, -0.1111, 0.8264, 0.21738, -0.63776, -0.074874
, -1.713, -0.8827, -0.0073058, -0.37623, -0.50209, -0.58844
, -0.24943, -1.0425, 0.27678, 0.64142, -0.64605, 0.43559
, -0.37276, -0.0032068, 0.18744, 0.30702],
[-0.79435, 0.73221, 0.30758, -0.55326, -1.0142, 0.71268
, -0.098544, 0.34649, 0.36471, -0.95308, 0.07392, 0.5812
, 0.47464, 0.59906, 0.19861, -0.16305, -0.21514, 0.23786
, -0.60277, 0.48853, 0.33604, 0.088385, -0.13442, -0.79827
, -0.24704, 0.76663, -0.89623, -0.7487, 0.26887, 0.39853
, -0.52541, 0.85086, 0.28696, -0.37791, 0.51659, 1.2911
, 0.14688, 0.39131, 0.0037607, -0.72696, 0.15202, -0.2881
, -0.9256, -0.82654, -0.043089, 0.13493, -0.70459, -0.069904
, -0.50487, -0.96548, 0.056805, -0.38206, -0.15625, 0.49787
, -0.60557, -1.0282, -0.13822, 0.067628, 1.0677, -0.31713
, 0.11543, 1.2516, -0.075387, -0.99882, 0.70076, 0.35112
, 0.32259, -0.42263, 0.39833, -0.68835, -0.48281, -0.46954
, 0.45839, 0.31533, -0.72773, 0.76295, -1.1747, -0.55477
, 0.43251, 0.70465, 0.12924, -0.48052, -1.1638, 0.56234
, -0.79682, -0.65032, 0.39826, 0.17965, -0.53933, 0.65524
, 0.59738, -0.73527, 0.66793, -0.58127, -1.1326, -1.0082
, 0.0025887, -0.32335, 0.69898, 0.26452],
[0.12728, 0.33918, 0.047297, -0.59545, -0.50807, -0.1497, 0.30002
, 0.28764, -0.45, -0.98081, -1.3502, 0.64971, 0.7544, -0.23777
, 0.80742, 0.59327, 0.41876, 0.070505, 0.026816, 0.2541, 0.72081
, 0.39636, 0.7526, -0.94346, 0.61644, 0.6186, -0.67133, -0.091559
, -0.3541, 0.65173, -0.049964, -0.12267, 0.20019, -0.097971, 0.25479
, 0.26575, -0.23218, 0.53218, -0.055243, -0.252, 0.31309, -0.07482
, -0.045389, -0.30998, -0.20094, -0.10533, 0.55923, -0.25827, -0.54868
, -0.57999, -0.67253, -0.034722, 0.27119, 0.82289, -0.71094, 0.02069
, -1.1679, 0.039353, 0.12002, -0.31662, 1.4502, 0.91076, 1.1657
, -0.21669, 1.3254, 0.8902, -0.43057, -0.77069, 0.90518, -0.38127
, -0.28038, 0.35435, 0.22964, -0.21386, 0.32834, 0.14031, 0.076089
, 0.74345, -0.3713, 0.52666, 0.6998, 0.64682, -0.51838, 0.30539
, -0.67797, -0.3944, 0.31669, 0.2097, 0.46173, 0.64263, -0.1257
, -0.82134, 0.37247, -0.39669, -1.2945, -0.73274, -0.72885, 0.26961
, 0.74588, -0.80255],
[-3.8032e-01, 1.5292e+00, -9.4527e-01, -1.3887e-03, 5.5101e-01, 4.8126e-01
, -1.2502e-01, 3.8229e-01, -7.9148e-01, -6.3399e-01, -1.0941e+00, -1.6552e-01
, 5.9088e-01, 5.0750e-01, 5.2301e-01, 4.2854e-01, 2.7123e-01, 5.0605e-01
, 6.1477e-01, -2.6955e-01, -6.3666e-01, -2.3305e-01, 3.3131e-01, -1.3382e-01
, 1.1594e+00, 1.4189e+00, 4.1742e-01, -4.4100e-01, -1.1032e-01, 1.1094e-02
, -6.1099e-01, -3.6861e-01, 1.4686e+00, -1.5860e-01, -3.8299e-01, 5.8125e-01
, -1.3489e-02, -1.0072e-02, -3.1125e-01, -8.7364e-01, 3.4630e-01, -4.5256e-01
, 1.7114e-01, -3.9858e-01, 1.0024e+00, 1.2011e-01, -3.9076e-01, 2.6316e-01
, -3.6461e-02, -1.0930e-01, -1.8831e-01, -7.0645e-01, 1.9801e-01, 4.5763e-01
, -2.3255e-01, -8.2976e-01, -1.2711e+00, 5.6615e-01, 1.0101e+00, 3.0987e-01
, 1.8205e-01, 6.0414e-01, -4.3705e-01, 9.5937e-01, 4.3737e-01, -1.7839e-01
, 8.1172e-01, -1.5144e-01, -1.3604e-01, -5.9853e-01, -1.8106e-01, 1.2243e-01
, -4.9549e-01, -9.0807e-01, 5.2064e-01, 1.5329e-01, -1.1036e-01, 1.0470e-01
, -4.8725e-01, 3.3595e-01, 1.0709e-01, 2.2925e-01, -2.8080e-01, -4.2324e-01
, -1.1206e-01, -4.0746e-01, 1.1435e-01, -7.2138e-01, 5.3190e-01, 9.7837e-01
, 7.8681e-02, -6.6394e-02, 2.0221e-01, -6.5273e-01, -5.9822e-01, 5.8856e-02
, -1.0303e+00, -5.6288e-01, 3.1628e-01, -1.2977e-01],
[-6.1040e-01, 2.8971e-01, 6.2573e-01, -6.5753e-01, -5.9586e-01, 7.6895e-02
, -6.0556e-01, 1.0634e-02, 3.4783e-01, -4.3325e-01, 1.3869e-01, 2.2622e-01
, -8.0813e-02, -2.4775e-01, -1.0613e-01, -5.5539e-01, -5.8195e-01, 2.1491e-01
, 2.8707e-02, 5.4337e-01, 5.3498e-01, 8.0320e-01, 2.7518e-01, -2.8186e-01
, -7.0555e-02, 3.5080e-01, -1.0586e+00, -1.0073e+00, -1.7399e-02, 1.0048e-01
, 6.8697e-02, 4.5259e-01, -2.3724e-01, -3.5439e-01, 2.2933e-01, 5.2986e-01
, -8.8526e-02, 1.7798e-01, -3.6976e-01, -4.7994e-02, -1.3154e-01, -2.5357e-01
, -1.7017e-01, -4.8352e-01, -3.6778e-01, -5.5602e-01, 5.8723e-01, 4.0939e-01
, 3.4807e-01, -8.4191e-01, 2.1293e-01, -4.0240e-01, 6.3245e-02, 1.2009e+00
, -3.1677e-01, -2.4433e+00, 3.6761e-01, 9.1433e-02, 1.5495e+00, 1.1378e-02
, 5.8241e-02, 1.3236e+00, -4.8752e-01, 1.7317e-01, 6.7283e-01, -2.3639e-01
, 8.8461e-01, 1.5139e-01, -1.2190e-01, -1.5251e-01, -1.9425e-01, -6.2059e-01
, 5.4896e-02, 1.7419e-03, 3.9242e-01, -2.7105e-01, -4.6918e-01, -6.7190e-02
, -5.2223e-01, 2.0321e-02, 8.1756e-01, -2.7581e-01, -6.5014e-01, -1.5146e-01
, -2.0179e+00, -5.1833e-01, -5.5365e-02, 1.5961e-01, -9.3043e-01, -4.8969e-01
, -3.3129e-02, -1.0458e-01, -1.8781e-01, -1.0454e-02, -4.2382e-01, -2.7874e-01
, -4.5546e-01, -3.5054e-01, 5.2370e-02, 3.4214e-01],
[-0.79435, 0.73221, 0.30758, -0.55326, -1.0142, 0.71268
, -0.098544, 0.34649, 0.36471, -0.95308, 0.07392, 0.5812
, 0.47464, 0.59906, 0.19861, -0.16305, -0.21514, 0.23786
, -0.60277, 0.48853, 0.33604, 0.088385, -0.13442, -0.79827
, -0.24704, 0.76663, -0.89623, -0.7487, 0.26887, 0.39853
, -0.52541, 0.85086, 0.28696, -0.37791, 0.51659, 1.2911
, 0.14688, 0.39131, 0.0037607, -0.72696, 0.15202, -0.2881
, -0.9256, -0.82654, -0.043089, 0.13493, -0.70459, -0.069904
, -0.50487, -0.96548, 0.056805, -0.38206, -0.15625, 0.49787
, -0.60557, -1.0282, -0.13822, 0.067628, 1.0677, -0.31713
, 0.11543, 1.2516, -0.075387, -0.99882, 0.70076, 0.35112
, 0.32259, -0.42263, 0.39833, -0.68835, -0.48281, -0.46954
, 0.45839, 0.31533, -0.72773, 0.76295, -1.1747, -0.55477
, 0.43251, 0.70465, 0.12924, -0.48052, -1.1638, 0.56234
, -0.79682, -0.65032, 0.39826, 0.17965, -0.53933, 0.65524
, 0.59738, -0.73527, 0.66793, -0.58127, -1.1326, -1.0082
, 0.0025887, -0.32335, 0.69898, 0.26452],
[-0.64334, 0.59266, 1.0349, -0.034131, 0.33263, -0.41222, -0.14829
, 0.25417, -0.17114, -0.053586, -0.20722, -0.39727, -0.30498, -0.048827
, -0.01901, 0.091807, 0.20736, -0.15835, -0.49164, -0.36594, 0.24915
, -0.26878, -0.039464, -0.27432, -0.55732, 0.61974, -0.61192, -0.010287
, 0.28992, -0.21781, 0.95627, 0.82756, 0.74257, 0.61147, -0.29291
, 1.2471, -0.41853, -0.50265, 0.077683, 1.1172, -1.4755, -0.87515
, 0.92085, 0.71645, -0.52272, -0.91533, 0.016309, -0.21561, -0.57944
, -1.3095, 0.47804, -0.45108, 0.80353, 1.286, -0.042785, -2.3869
, 0.69866, -0.29288, 0.80929, 0.31218, -0.1893, 0.21965, 0.46042
, -0.7871, 0.53793, 0.045084, 0.199, 0.72672, -0.38028, 0.26402
, 0.077038, 0.025636, -0.9315, -0.68583, -0.18519, 0.026895, -0.11008
, 0.35026, -0.79036, 0.30193, 0.64517, -0.44596, -0.14865, -0.52965
, -1.2032, -0.26656, 0.17308, 0.35856, -0.20222, -1.1401, -0.11777
, -0.37037, -0.075284, 0.26164, -1.0166, -0.36246, -0.039511, 0.15483
, 0.29981, 0.37799],
[-1.9006e-01, 3.1120e-01, 4.7171e-01, -2.8056e-01, -4.8337e-01, 6.8553e-02
, -4.5067e-01, 1.1173e-01, 6.6101e-02, -7.1041e-01, -2.4859e-01, 1.3370e-01
, -1.3752e-01, -1.5745e-01, 4.0046e-01, 5.2179e-02, -2.0072e-01, -7.5249e-03
, 2.4160e-01, 6.9304e-01, 5.8341e-01, 9.3729e-01, -9.5317e-02, -6.1119e-01
, 1.5453e-01, 2.0619e-01, -5.8002e-01, -8.1164e-01, -2.4534e-01, 1.0759e-01
, -4.7973e-02, 6.4102e-02, 2.2681e-01, -3.3923e-01, -1.5209e-01, 3.1977e-01
, -3.7060e-02, 3.6081e-01, 5.7981e-01, -8.0079e-02, -1.5822e-01, -5.8878e-01
, -6.2072e-02, -7.0349e-01, -2.6627e-01, -2.4602e-01, 4.5015e-01, 2.5519e-01
, 4.8353e-01, -7.6818e-01, -2.2812e-01, -2.9609e-01, 1.1409e-01, 9.1713e-01
, -2.9223e-01, -3.1109e+00, -5.4891e-02, -3.6824e-01, 1.1675e+00, 5.4894e-02
, 1.3308e-01, 1.2725e+00, -1.1196e+00, 3.5718e-01, 6.8855e-01, -5.2246e-01
, 4.9855e-01, 6.1230e-02, 4.1514e-01, -5.0192e-02, -1.1963e-02, -2.6769e-01
, -1.3192e-01, -1.5594e-01, 1.9538e-01, 8.1034e-01, -1.5480e-01, 4.1017e-02
, -5.1727e-01, 1.9496e-01, 4.7015e-01, -1.1347e-01, -1.6959e-01, -7.4851e-03
, -1.5788e+00, -1.7431e-01, 3.3742e-01, 4.5661e-02, -2.1704e-01, -3.9708e-01
, -2.9676e-03, -2.2496e-01, -2.1707e-01, 6.9045e-02, -6.8760e-01, -4.2580e-01
, -5.8658e-01, -5.3615e-01, 6.2931e-02, 6.1613e-01],
[0.1472, -0.74926, 0.71394, 0.50613, -0.48654, -0.55406
, -0.24556, 0.45502, -0.24201, 0.045856, -0.21361, 0.31994
, -0.055064, -0.41279, 0.82932, 0.35905, -1.0126, -0.32741
, 0.31357, -0.2557, 0.59086, -0.081674, -0.44165, 0.25708
, 0.097546, 0.47901, -0.30838, 0.39091, 0.53529, -0.37325
, -0.20628, 0.16935, -0.49781, -0.41903, -1.0872, 0.073446
, 0.62999, -0.56862, -0.29221, -0.36879, -0.33794, 0.37359
, -0.02864, 0.17844, -0.28942, 0.43309, 0.69843, 0.4135
, 0.305, -0.64137, 0.32536, -0.85361, 0.37063, 0.24458
, 0.24942, -0.88577, 0.49808, -0.26253, 0.40472, 0.10354
, -0.20208, 0.24582, 0.29527, -0.33242, 0.75299, -0.76986
, 0.12593, -0.84228, -0.15694, -0.54902, -0.64156, -0.0063947
, -0.71797, -0.66911, 0.35953, -0.53917, -0.44646, 0.19282
, 0.20925, -0.27101, 0.34209, -0.39717, 0.085393, 0.050575
, -0.424, -0.99114, 0.51179, -0.053704, -0.40943, -1.0351
, 0.53723, 0.59613, -0.93641, 0.38519, 0.14502, -0.68173
, -0.10823, -0.12622, -0.25814, -0.25356],
[0.058599, 0.32109, 0.13023, -0.082613, -0.21887, -0.36561, -0.61741
, -0.38255, -0.15554, -1.2769, -0.33592, 0.72739, 0.83975, 0.15045
, 0.15586, 0.58761, 0.76943, 0.050015, -0.74405, 0.27222, -0.17157
, -0.0263, -0.41211, 0.2121, -0.063788, 0.72447, -1.3064, 0.18043
, -0.23386, -0.087528, 0.65693, -0.31243, 0.027384, 0.067259, -0.088452
, -0.3833, 0.47839, 0.27438, 0.6315, -0.43812, 0.42713, -0.16148
, -0.51099, -0.11995, -0.021659, 0.34015, 0.35593, 0.55595, -0.84954
, -0.52109, -0.3573, -0.32797, 0.1428, 0.87888, 0.25859, -2.0342
, -0.016375, -0.36715, 0.78913, 0.013592, 0.68704, 1.2293, -0.29089
, 0.34655, 0.95929, 0.044547, 0.20548, 0.34915, 0.074704, 0.17018
, -0.89121, -0.42363, -0.43282, 0.1571, -0.014452, 0.070268, -0.90682
, -0.19523, -0.94521, -0.14821, 0.46188, -0.20295, -0.28233, -0.32566
, -0.44746, -0.41196, -0.38912, -0.63425, 0.83687, 0.48267, 0.097352
, -0.45653, 0.64334, -0.79794, -0.35467, 0.20268, -0.68918, 0.11792
, 0.36543, 0.22123],
[-0.69634, 0.42985, -0.49358, 0.3865, -0.36046, 1.0034, -0.77896
, 0.63168, -0.17716, -0.4821, 0.15906, 0.58443, 0.35491, -0.40479
, -0.34624, -0.23395, -0.079658, 0.63091, -1.5473, -0.070053, 0.18411
, -0.013936, -0.57734, 0.47626, 0.34182, -0.062102, -0.76948, 0.029682
, 0.62975, -0.033092, -0.73824, -0.35622, -0.55253, 0.45102, -0.25186
, 0.90617, -0.041602, 0.026572, 0.47098, -0.91099, -0.23537, 0.84552
, -0.61004, -0.49121, -1.2357, 0.23482, 0.16825, -0.083965, 0.097143
, -0.60303, -0.10492, -0.22775, 0.030855, 0.57033, -0.37246, 0.61821
, 0.50403, 0.092966, 0.20154, -0.088136, -0.017448, 0.63977, -0.09851
, -0.30846, -0.36724, 0.68269, 0.4293, -0.50889, -0.18819, -0.37411
, 0.025594, -0.22269, 1.1093, -0.13815, -0.55312, 0.14661, -0.3644
, -0.73527, 0.68179, -0.1662, -0.32343, 0.065271, -0.21674, 0.42364
, 0.43569, 0.01023, 0.15533, 0.11457, -0.40894, 0.34447, 0.077626
, -0.16337, 0.12356, 0.3928, 0.19189, -0.48342, 0.029379, -0.70546
, 0.32409, 0.29724],
[-0.29581, 0.30855, 0.82131, 0.0085845, -0.66401, -0.069684
, 0.085665, -0.46486, 1.179, 0.03124, -0.16966, 0.19938
, 0.24664, 0.16489, -0.35447, -0.33572, -0.44905, 0.21919
, -0.036618, -0.65902, -0.43739, -0.09481, -0.031618, -0.15844
, 0.014121, 0.038115, 0.18901, -0.61176, -0.59838, 0.071352
, -0.36706, 0.14968, 0.39045, 0.049271, 0.10153, 0.13062
, -0.038567, 0.2845, 0.51043, -0.22678, -0.74117, 0.32308
, 0.26895, -0.77992, -0.28285, 0.09795, 0.527, -0.016245
, -0.71906, -0.32505, 0.63978, -0.21855, -0.66017, 1.2624
, 0.32059, -2.2355, -0.091599, -0.10159, 1.6238, 0.73467
, 0.020861, 0.24514, -0.327, -0.16384, 1.3841, 0.46869
, 0.58052, -0.35346, 0.41557, -0.058828, -0.28866, -0.020676
, -0.29129, 0.063786, 0.70216, -0.2204, -0.089771, 0.29046
, -0.86696, 0.74243, 0.57275, -0.14929, -0.73445, -0.21592
, -1.772, 0.17907, -0.36801, -0.17197, -1.1736, -0.55531
, -0.59886, 0.2586, -0.55728, -0.088817, 0.21041, 0.46815
, 0.079753, -0.35043, 0.69074, -0.058652],
[-2.4149e-01, 4.3733e-01, -3.2623e-01, -1.8897e-01, -2.9305e-01, -2.8223e-01
, -3.7429e-01, 4.7172e-01, -6.1045e-01, -1.7059e-01, -5.5280e-02, 7.4618e-01
, 6.1522e-01, 2.9321e-01, 2.3069e-01, -4.0869e-01, 2.4888e-01, -4.8797e-01
, 4.8047e-04, -5.4519e-01, 5.2164e-01, 7.3132e-01, -1.0567e-03, 1.0601e-01
, 8.4546e-01, 4.9301e-01, -9.2694e-01, -5.6454e-01, -7.3725e-02, 8.7184e-02
, -1.7091e-01, 2.9569e-01, 4.1866e-01, -3.5302e-01, 3.3314e-01, 2.3985e-01
, 5.1223e-01, 3.1759e-01, 3.6975e-01, 5.2439e-01, -1.5201e-01, -5.5230e-01
, 4.1703e-01, -1.3236e-01, 9.5632e-02, -4.7920e-01, 8.2726e-01, -4.1113e-01
, 2.4488e-01, -1.1884e-01, 2.5559e-01, -5.5004e-01, 4.8753e-01, 1.2109e+00
, 4.1641e-01, -2.8652e+00, 2.6831e-01, -9.4585e-02, 1.8959e+00, 4.3377e-01
, -1.7049e-01, 8.2421e-01, 1.0373e-01, -1.8124e-01, 6.3302e-01, -2.8780e-02
, 6.6535e-01, 1.2374e-01, -8.8930e-02, -1.8875e-01, 2.6731e-01, 1.3841e-01
, -1.1260e-01, -3.4640e-01, -4.4674e-02, -4.5838e-01, -6.6339e-01, -1.8257e-02
, -8.9461e-01, -2.7158e-01, 7.0234e-01, -7.7111e-02, -3.5251e-01, 1.5090e-01
, -1.7293e+00, 2.0593e-01, -2.3115e-01, 7.9508e-02, -9.2586e-02, -1.1143e-01
, -1.8736e-01, 3.2542e-01, -3.8972e-01, -3.7078e-01, -6.5679e-01, -8.2912e-02
, -5.1918e-01, -3.4876e-01, 2.3791e-01, 2.1206e-01],
[0.20685, 0.22292, -0.33488, 0.38882, -0.3302, 0.84261
, -0.67919, -0.39023, -0.29869, -1.1832, 0.25024, -0.12468
, 0.16641, -0.096326, 0.044293, 0.036477, 0.073808, 0.47124
, 0.50309, -0.18468, 0.69457, 0.18977, 0.29172, -0.67525
, 0.88308, 0.22486, -0.39175, 0.43618, 0.34951, 0.37277
, 0.34881, -0.32953, 0.038894, 0.21444, 0.067972, 0.23505
, 0.77618, -0.66022, 0.3098, 0.058035, 0.22192, 0.6226
, -0.29997, 0.43059, 0.94422, -0.66305, 0.30799, -0.023856
, -0.026733, -0.5571, 0.24467, -0.54794, 1.0783, 0.5847
, 0.079197, -1.7291, -0.41867, 0.2166, 0.99454, 0.0082021
, 0.56054, 0.96743, -0.23923, -0.54929, 0.8798, -0.39043
, 0.34745, 0.1736, -0.51619, 0.75369, 0.029741, -0.52674
, 0.16762, -0.039339, -0.38028, 0.40275, 0.11921, 0.15769
, -0.14727, -0.87732, -0.20805, -0.20563, 0.38439, 0.43164
, -1.4708, 0.22218, -0.052566, 0.42501, -0.07543, -0.065633
, 0.11181, -0.28626, 0.18907, 0.94487, -0.33292, -0.041871
, -0.27163, -0.10011, -0.4411, -0.38403],
[2.3302e-01, 3.8258e-01, 8.9732e-01, -6.0346e-01, -6.8923e-01, 2.2225e-01
, 1.5507e-01, -1.8096e-01, 6.2266e-01, -3.6591e-01, 3.2201e-01, -4.8163e-02
, 1.3591e-01, -5.8990e-02, -4.2783e-06, 1.9496e-01, -2.5979e-01, 5.5862e-01
, -1.6651e-01, 8.4867e-01, 3.6100e-01, 4.5194e-01, -5.4684e-01, -8.8664e-01
, 3.5322e-01, 6.0532e-01, -2.8882e-01, -5.0539e-01, 4.2566e-02, -4.5517e-01
, -2.0077e-01, 4.6050e-01, 7.1080e-01, -3.2255e-01, -2.4540e-01, -1.0679e-02
, -6.5341e-01, 2.0207e-01, 6.1073e-01, -8.3154e-01, -2.6067e-01, 1.3994e-01
, 2.8348e-01, -5.5256e-01, -9.0848e-01, 1.9526e-02, 5.2753e-01, -2.2129e-01
, -5.3881e-01, -1.1192e+00, 2.4878e-01, -1.4016e-01, 1.6105e-02, 7.6864e-01
, 7.1804e-02, -2.4239e+00, 3.4912e-02, 6.8972e-01, 1.4261e+00, 4.4535e-01
, -1.7790e-01, 8.9169e-01, -5.1200e-01, -2.9202e-01, 1.2092e+00, 3.5658e-02
, 8.0488e-01, 2.1017e-01, -1.5366e-01, -4.5057e-01, 3.7093e-01, -4.0628e-01
, 4.0000e-02, 1.4657e-01, 2.7272e-01, 1.3961e-01, 6.1806e-03, -2.9560e-01
, 1.7627e-02, 3.6324e-01, 8.7452e-02, -6.0443e-02, -3.1265e-01, -1.1006e-01
, -1.5961e+00, 7.5077e-02, -3.1300e-01, -2.5681e-01, -3.3984e-01, -6.8656e-01
, -8.5334e-02, -2.7824e-01, 1.3657e-01, -7.9241e-02, -4.6558e-01, 2.4889e-01
, -7.4093e-02, -2.9610e-01, -1.4922e-02, 8.5423e-01],
[-1.2743e-01, 4.3527e-01, 8.6000e-01, 1.5689e-01, -4.3569e-01, 1.1583e-01
, -2.1007e-01, -5.0635e-01, 6.4092e-01, -4.6156e-01, -1.9420e-01, 5.8436e-01
, 6.9742e-02, -1.1545e-01, -3.2735e-01, -3.9723e-01, -2.0121e-01, 5.8893e-01
, -5.7793e-01, -1.9917e-01, -2.9232e-02, -3.0948e-01, -3.2966e-01, -4.6377e-01
, 1.6894e-01, -2.1959e-01, 2.2170e-01, -3.2093e-01, -2.8745e-01, 1.0514e-01
, -3.0371e-01, 1.6153e-01, 3.8239e-01, -1.1834e-01, -2.6868e-01, 1.1647e-01
, -2.6755e-01, 8.7568e-02, 3.8210e-01, -2.8546e-01, -7.4351e-01, -2.4122e-01
, 4.6977e-01, -4.8640e-01, -8.4232e-01, -5.2109e-01, 3.4324e-01, -2.6099e-01
, -7.5433e-01, -7.7240e-01, 9.5811e-01, 1.0273e-01, 2.2422e-01, 1.2683e+00
, -1.8940e-01, -1.8174e+00, 1.6384e-01, -4.3998e-01, 1.4220e+00, 4.3426e-01
, -1.5507e-01, 6.2946e-01, -6.7967e-04, -3.8197e-01, 1.5199e+00, -2.0684e-01
, 3.8709e-01, 2.5181e-01, -5.3548e-01, 3.5606e-01, -2.5254e-02, 2.4610e-01
, -1.9801e-01, -2.8623e-01, -1.2536e-01, 1.6036e-01, -2.4481e-01, -3.7300e-01
, -9.3496e-01, -6.1637e-02, 3.1735e-01, -5.1726e-01, -2.8423e-02, -6.4262e-01
, -1.5638e+00, -4.1716e-01, -3.1100e-01, 3.9429e-01, 5.3170e-02, -7.1948e-01
, -2.8150e-01, 2.2780e-01, -3.2269e-01, -1.6102e-01, -1.7337e-01, 6.0107e-01
, 1.9735e-02, -2.4033e-01, 3.7498e-01, 6.7258e-01],
[-0.57306, 0.25687, 0.49477, 0.36484, -0.26337, 0.55617
, -0.21334, 0.15467, 0.18497, 0.37596, -0.51661, -0.25104
, 0.51243, 0.43471, 0.35942, -0.29295, 0.32112, -0.19484
, 0.035392, 0.085374, 0.25988, -0.42721, -0.80673, 0.3342
, 0.7198, -0.28185, 0.86516, -0.44575, 0.74551, -0.34252
, -0.065908, 0.11546, 0.15942, -0.38512, 0.63627, -0.1477
, -0.54938, -0.14532, -0.23708, -0.2479, 0.67518, -0.50745
, 0.24784, 0.19225, 0.31172, 0.14891, -0.089564, 0.25932
, -0.12003, 0.37353, 0.16243, 0.70705, 0.78489, 0.56606
, -0.88018, -0.11696, 0.063664, 0.63159, 0.30215, 1.0165
, -0.40015, -0.623, -0.69339, 0.43775, 0.045461, -0.40482
, -0.66249, -0.45242, 0.58096, 0.0086489, -0.88649, 0.21438
, 0.0074026, -0.76973, 0.5328, 0.48181, 0.82483, -0.083315
, -0.98326, -0.5464, 0.34003, -2.2002, -0.55954, -0.84781
, -0.31704, 0.17179, 0.23368, -0.8381, 0.47945, 1.1727
, -1.2655, 1.1252, 0.33894, 0.15648, 1.1288, -0.20941
, -0.33243, -0.9095, -0.27798, 0.24324],
[-0.029785, 0.086459, 0.64196, -0.26033, -0.49155, 0.19763
, -0.20132, -0.062676, 0.43697, -0.12687, 0.47302, 0.59099
, 0.019121, 0.15612, 0.049087, -0.10333, -0.56733, 0.55924
, -0.40351, 0.85564, 0.11834, 0.035305, -0.28665, -1.1512
, -0.2064, 0.3473, -0.64901, -1.0647, 0.11578, -0.17317
, -0.099097, 0.61244, 0.19025, -0.19077, 0.16614, 0.42234
, -0.11202, -0.15759, -0.38879, -0.32297, -0.17964, 0.22422
, 0.02739, -0.65288, -0.93387, -0.2776, 0.4697, -0.17291
, -0.51162, -1.6382, 0.89075, 0.25091, 0.14141, 0.70431
, -0.12055, -2.0169, 0.17555, 0.23129, 1.3212, 0.196
, -0.28592, 1.3289, -0.82781, -0.27755, 0.64261, -0.060212
, 1.187, 0.27061, -0.38198, -0.20732, -0.0095631, -0.38121
, 0.047486, -0.11944, 0.6516, 0.24754, -0.10522, -0.36625
, -0.92738, 0.52657, 0.23004, -0.28603, -0.60805, -0.075451
, -1.2444, -0.222, 0.15635, 0.18156, -0.040432, -0.37308
, 0.079419, -0.20566, 0.049851, -0.30412, -0.14789, -0.44173
, -0.36753, 0.077098, 0.12672, 0.34821]], dtype='float32')

b_p_ten = K.variable (value=b_padding, dtype='float32')

b_fw = np.array([
         [0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]
         ], dtype='float32')

b_fw_ten = K.variable (value=b_fw, dtype='float32')

b_bw = np.array([
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]
        ], dtype='float32')

b_bw_ten = K.variable (value=b_bw, dtype='float32')

features = [a_padding, b_padding]
fw_tens = [a_fw, b_fw]
bw_tens = [a_bw, b_bw]
dataset = [features, fw_tens, bw_tens]


label = np.concatenate((a_padding, b_padding), axis=1)
print('shape_label: ', label.shape)


In [ ]:
builder = ModelBuilder(input_enc_dim=100, edge_dim=19, input_dec_dim=1000)
graph_emb_layers, graph_embedding_encoder_states = builder.GraphEmbeddingEncoderBuild(hops=3)



seq_embedding = Embedding(input_dim=1680, output_dim=100, input_length=1000)(builder.get_decoder_inputs())

decoder_lstm = LSTM(200, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(seq_embedding, initial_state=graph_embedding_encoder_states)
decoder_dense = Dense(100, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)


#seq_embedding = Embedding(input_dim=1680, output_dim=100, input_length=1000)(builder.get_decoder_inputs())
#graph_decoder_model = builder.GraphEmbeddingDecoderBuild(   embedding_layer=seq_embedding,
#                                                            prev_memory_state=graph_embedding_encoder_states[0],  
#                                                            prev_carry_state=graph_embedding_encoder_states[1],
#                                                            num_dec_tokens=1680)


'''graph_decoder_model = builder.MakeModel(layers=[decoder_outputs])
graph_decoder_model.compile(loss='mean_squared_error',
                        optimizer='rmsprop',
                        metrics=['mae', 'acc'])

builder.Summary(graph_decoder_model)

builder.Plot(model=graph_decoder_model, file_name='graph_decoder_model.png')'''

model = Model(builder.get_inputs(), decoder_outputs)
builder.Summary(model)
builder.Plot(model=model, file_name='model_graph.png')

In [ ]:
'''graph_emb_model.fit(x=[features[0], fw_tens[0], bw_tens[0]],
                    y=label,
                    epochs=25)'''